In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima


In [2]:
load_path = 'processed_data/data_2006_2023.csv'

In [3]:
load_data = pd.read_csv(load_path)

In [4]:
load_data

,loadConsumption,date,datetime,Hour,Year
0,10215.0000,2006-01-01,2005-12-31 23:00:00,0,2006
1,9979.0000,2006-01-01,2006-01-01 00:00:00,1,2006
2,9460.0000,2006-01-01,2006-01-01 01:00:00,2,2006
3,8833.0000,2006-01-01,2006-01-01 02:00:00,3,2006
4,8525.0000,2006-01-01,2006-01-01 03:00:00,4,2006
...,...,...,...,...,...
157767,12673.4650,2023-12-31,2023-12-31 19:00:00,20,2023
157768,12366.1950,2023-12-31,2023-12-31 20:00:00,21,2023
157769,12143.1775,2023-12-31,2023-12-31 21:00:00,22,2023
157770,11993.1200,2023-12-31,2023-12-31 22:00:00,23,2023


In [5]:
# Check for stationarity using Augmented Dickey-Fuller test
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    if result[1] < 0.05:
        print("Data is stationary")
    else:
        print("Data is NOT stationary")

In [6]:
adf_test(load_data["loadConsumption"])


MemoryError: Unable to allocate 49.3 MiB for an array with shape (41, 157695) and data type float64

In [ ]:
# Load data (ensure the date column is in datetime format)
df = load_data.copy()  
df["date"] = pd.to_datetime(df["date"])

In [ ]:
# Aggregate hourly data to daily
df_daily = df.groupby("date")["loadConsumption"].sum().reset_index()

adf_test(df_daily["loadConsumption"])


### ARIMA 
On daily load consumption

In [ ]:
# Split into train-test sets (e.g., last 365 days for testing)
train_size = len(df_daily) - 365
train, test = df_daily[:train_size], df_daily[train_size:]

# Use Auto-ARIMA to find the best (p, d, q) parameters
auto_model = auto_arima(train["loadConsumption"], seasonal=False, stepwise=True, suppress_warnings=True)
print(f"Optimal ARIMA parameters: {auto_model.order}")

# Train ARIMA model using the best parameters
p, d, q = auto_model.order
model = ARIMA(train["loadConsumption"], order=(p, d, q))
model_fit = model.fit()

# Make predictions
forecast = model_fit.forecast(steps=len(test))

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

# Calculate R^2 score
r2 = r2_score(test["loadConsumption"], forecast)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((test["loadConsumption"] - forecast) / test["loadConsumption"])) * 100

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(test["loadConsumption"], forecast)

# Calculate RMSE
rmse = np.sqrt(np.mean((forecast - test["loadConsumption"])**2))


# Print additional metrics
print(f"R^2 Score: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse}")


In [ ]:
# Plot actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(train["date"], train["loadConsumption"], label="Train Data")
plt.plot(test["date"], test["loadConsumption"], label="Actual Test Data", color="blue")
plt.plot(test["date"], forecast, label="ARIMA Forecast", color="red", linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Electricity Load")
plt.title("ARIMA Forecast vs Actual Load")
plt.legend()
plt.show()


The model is not capturing any pattern

### weather data

In [ ]:
weather_data_path = 'processed_data/weather.csv' 
weather_data = pd.read_csv(weather_data_path)

In [ ]:
weather_data.head()

In [ ]:
weather_data["Date"] = pd.to_datetime(weather_data["Date"])

In [ ]:
# Merge temperature data
df_merged = pd.merge(df_daily, weather_data, left_on='date', right_on='Date', how='inner')

df_merged.drop(columns=['Date'], inplace=True)

In [ ]:
df_merged["is_weekend"] = df_merged["date"].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)


In [ ]:
import holidays
# Define holidays for the Netherlands
nl_holidays = holidays.NL(years=df_merged["date"].dt.year.unique())

# Create a binary column for holidays (1 = holiday, 0 = non-holiday)
df_merged["is_holiday"] = df_merged["date"].apply(lambda x: 1 if x in nl_holidays else 0)


In [ ]:
df_merged.head()

In [ ]:
#df_merged.to_csv('df_merged.csv', index = False)

In [ ]:
import seaborn as sns

# Drop redundant columns
df_cleaned = df_merged.drop(columns=[ "WeekKey",
                                     "day_of_week", "HDMaxPrecipitation", 
                                     "HDMaxMeanWindspeed", "HDMinMeanWindspeed"])

# Convert date column to datetime format
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"])

numeric_cols = df_cleaned.select_dtypes(include=["number"])

# Compute correlation matrix
corr_matrix = numeric_cols.corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Matrix of Electricity Load and Explanatory Variables")
plt.show()


In [ ]:
df_merged.columns

## SARIMAX (weather data, rolling averags, holiday feature)

In [ ]:
# Convert 'date' to datetime format
df_merged["date"] = pd.to_datetime(df_merged["date"])

# Drop unnecessary columns
columns_to_drop = [
    "WeekKey","Year", "Month",'day_of_week',
    "MaxHourlyPrecipitation", "HourlyMaxTemperature", "HDMaxTemperature",  
    "HDMaxMeanWindspeed", "HDMinMeanWindspeed", "MinHourlyMeanWindspeed"  
]

df_sarimax = df_merged.drop(columns=columns_to_drop)

# Convert categorical columns to numeric
df_sarimax.set_index("date", inplace=True)

df_sarimax["day_of_week"] = df_sarimax.index.weekday
df_sarimax["is_weekend"] = df_sarimax["is_weekend"].astype(int)
df_sarimax["is_holiday"] = df_sarimax["is_holiday"].astype(int)
df_sarimax["rolling_7d"] = df_sarimax["loadConsumption"].rolling(7).mean()
df_sarimax["rolling_30d"] = df_sarimax["loadConsumption"].rolling(30).mean()
df_sarimax = df_sarimax.fillna(method='bfill')  # Fill missing values


# Display structure after cleaning
df_sarimax.info()
df_sarimax.head()

 auto_arima helps in selecting the optimal ARIMA model by searching through different combinations of parameters and choosing the best one based on a metric like AIC (Akaike Information Criterion).

In [ ]:
from pmdarima import auto_arima

# Run Auto-ARIMA to find optimal parameters
auto_model = auto_arima(df_sarimax["loadConsumption"], seasonal=True, m=7, 
                        exogenous=df_sarimax[["day_of_week", "rolling_7d", "rolling_30d", "is_weekend", "is_holiday"]], 
                        stepwise=True, trace=True)

auto_model.summary()


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# Train-test split
train_size = int(len(df_sarimax) * 0.8)
train, test = df_sarimax.iloc[:train_size], df_sarimax.iloc[train_size:]

In [ ]:
# Define endogenous (target) and exogenous (features)
endog_train = train["loadConsumption"]
exog_train = train[["day_of_week", "rolling_7d", "rolling_30d", "is_weekend", "is_holiday"]]

endog_test = test["loadConsumption"]
exog_test = test[["day_of_week", "rolling_7d", "rolling_30d", "is_weekend", "is_holiday"]]

In [ ]:
# Use Auto-ARIMA parameters if available
best_p, best_d, best_q = auto_model.order  
best_P, best_D, best_Q, best_s = auto_model.seasonal_order  

In [ ]:
# Train SARIMAX model
sarimax_model = SARIMAX(endog_train, exog=exog_train, 
                        order=(best_p, best_d, best_q), 
                        seasonal_order=(best_P, best_D, best_Q, best_s),  
                        enforce_stationarity=False,
                        enforce_invertibility=False)

sarimax_fit = sarimax_model.fit()

# Forecast
forecast = sarimax_fit.forecast(steps=len(test), exog=exog_test)

# Prevent negative values
forecast = np.maximum(forecast, 0)


In [ ]:
#Model Evaluation _ no weather data

# RMSE
rmse_sarimax = np.sqrt(((forecast - endog_test) ** 2).mean())

# R^2 Score
r2_sarimax = r2_score(endog_test, forecast)

# Mean Absolute Percentage Error (MAPE)
mape_sarimax = np.mean(np.abs((endog_test - forecast) / endog_test)) * 100

# Mean Absolute Error (MAE)
mae_sarimax = mean_absolute_error(endog_test, forecast)

print(f" RMSE: {rmse_sarimax}")
print(f" R2: {r2_sarimax}")
print(f" MAPE: {mape_sarimax}")
print(f" MAE: {mae_sarimax}")

# Save metrics to a DataFrame
metrics_sarimax_df = pd.DataFrame({
    "Metric": ["RMSE", "R^2 Score", "MAPE", "MAE"],
    "Value": [rmse_sarimax, r2_sarimax, mape_sarimax, mae_sarimax]
})



In [ ]:
# Plot results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train["loadConsumption"], label="Train Data")
plt.plot(test.index, test["loadConsumption"], label="Actual Test Data", color="blue")
plt.plot(test.index, forecast, label="SARIMAX Forecast", color="red", linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Electricity Load")
plt.title("Improved SARIMAX Forecast vs Actual Load")
plt.legend()
plt.show()

In [ ]:
# Save to a CSV file
file_path_sarimax = "metrics/sarimax_metrics.csv"
metrics_sarimax_df.to_csv(file_path_sarimax, index=False)

In [ ]:
# Convert categorical columns to numeric
df_merged.set_index("date", inplace=True)

df_merged["day_of_week"] = df_merged.index.weekday
df_merged["is_weekend"] = df_merged["is_weekend"].astype(int)
df_merged["is_holiday"] = df_merged["is_holiday"].astype(int)
df_merged["rolling_7d"] = df_merged["loadConsumption"].rolling(7).mean()
df_merged["rolling_30d"] = df_merged["loadConsumption"].rolling(30).mean()
df_sarimax = df_sarimax.fillna(method='bfill')  # Fill missing values


auto_model_2 = auto_arima(df_merged["loadConsumption"], seasonal=True, m=7, 
                        exogenous=df_merged[["DailyPrecipitation", "DailyMeanTemperature",
                                             "HourlyMinTemperature", "day_of_week",
                                             "rolling_7d", "rolling_30d"]],
                        stepwise=True, trace=True)
#print(auto_model_2.summary())

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define endogenous (target) and exogenous (features)
endog_train = train["loadConsumption"]
exog_train = train[["DailyPrecipitation", "DailyMeanTemperature", "HourlyMinTemperature", 
                    "day_of_week", "rolling_7d", "rolling_30d"]]

endog_test = test["loadConsumption"]
exog_test = test[["DailyPrecipitation", "DailyMeanTemperature", "HourlyMinTemperature", 
                "day_of_week", "rolling_7d", "rolling_30d"]]

# Use Auto-ARIMA parameters if available
best_p, best_d, best_q = auto_model.order  
best_P, best_D, best_Q, best_s = auto_model.seasonal_order  

# Train SARIMAX model
sarimax_model = SARIMAX(endog_train, exog=exog_train, 
                        order=(best_p, best_d, best_q), 
                        seasonal_order=(best_P, best_D, best_Q, best_s),  
                        enforce_stationarity=False,
                        enforce_invertibility=False)

sarimax_fit = sarimax_model.fit()

# Forecast
forecast = sarimax_fit.forecast(steps=len(test), exog=exog_test)

# Prevent negative values
forecast = np.maximum(forecast, 0)

In [ ]:
# Evaluate performance
rmse_sarimax2 = np.sqrt(((forecast - endog_test) ** 2).mean())
r2_sarimax2 = r2_score(endog_test, forecast)
mape_sarimax2 = np.mean(np.abs((endog_test - forecast) / endog_test)) * 100
mae_sarimax2 = mean_absolute_error(endog_test, forecast)

print(f"RMSE: {rmse_sarimax2}")
print(f"R^2 Score: {r2_sarimax2:.4f}")
print(f"MAPE: {mape_sarimax2:.2f}%")
print(f"MAE: {mae_sarimax2:.4f}")

# Save metrics to a DataFrame
metrics_sarimax_df = pd.DataFrame({
    "Metric": ["RMSE", "R^2 Score", "MAPE", "MAE"],
    "Value": [rmse_sarimax2, r2_sarimax2, mape_sarimax2, mae_sarimax2]
})

# Save to a CSV file
file_path_sarimax = "metrics/sarimax_metrics(+weatherDate).csv"
metrics_sarimax_df.to_csv(file_path_sarimax, index=False)

print(f"Metrics saved to: {file_path_sarimax}")


In [ ]:
# Plot results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train["loadConsumption"], label="Train Data")
plt.plot(test.index, test["loadConsumption"], label="Actual Test Data", color="blue")
plt.plot(test.index, forecast, label="SARIMAX Forecast", color="red", linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Electricity Load")
plt.title("Improved SARIMAX Forecast vs Actual Load")
plt.legend()
plt.show()

#### 360 days of prediction

In [ ]:
sarimax_fit = sarimax_model.fit()

# Forecast for x days
forecast_horizon = 365  # Change days
forecast = sarimax_fit.forecast(steps=forecast_horizon, exog=exog_test[:forecast_horizon])

# Prevent negative values
forecast = np.maximum(forecast, 0)


In [ ]:
# Plot results
plt.figure(figsize=(12, 6))
plt.plot(train.index, train["loadConsumption"], label="Train Data")
plt.plot(test.index[:forecast_horizon], test["loadConsumption"].iloc[:forecast_horizon], label="Actual Test Data", color="blue")
plt.plot(test.index[:forecast_horizon], forecast, label="SARIMAX Forecast", color="red", linestyle="dashed")
plt.xlabel("Date")
plt.ylabel("Electricity Load")
plt.title("Improved SARIMAX Forecast vs Actual Load ({} Days)".format(forecast_horizon))
plt.legend()
plt.show()

In [ ]:
# Evaluate performance
rmse = np.sqrt(((forecast - endog_test.iloc[:forecast_horizon]) ** 2).mean())
r2_sarimax = r2_score(endog_test.iloc[:forecast_horizon], forecast)
mape_sarimax = np.mean(np.abs((endog_test.iloc[:forecast_horizon] - forecast) / endog_test.iloc[:forecast_horizon])) * 100
mae_sarimax = mean_absolute_error(endog_test.iloc[:forecast_horizon], forecast)

print(f"RMSE: {rmse}")
print(f"R^2 Score: {r2_sarimax:.4f}")
print(f"MAPE: {mape_sarimax:.2f}%")
print(f"MAE: {mae_sarimax:.4f}")

In [ ]:
# Save metrics to a DataFrame
metrics_sarimax_df = pd.DataFrame({
    "Metric": ["RMSE", "R^2 Score", "MAPE", "MAE"],
    "Value": [rmse, r2_sarimax, mape_sarimax, mae_sarimax]
})

# Save to a CSV file
file_path_sarimax = "metrics/sarimax_metrics(+weather,365).csv"
metrics_sarimax_df.to_csv(file_path_sarimax, index=False)